To create boolean search queries, we have Phi-3.5 generate a list of terms that we can combine into a search query.

In [200]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

def prompt_query(query):
    ''' Takes a text prompt and returns the binary search query to be used on academic databases. 
    Uses the keywords and = &, or = |. '''
    load_dotenv()
    api_key = os.getenv("AZURE_INFERENCE_CREDENTIAL", '')
    if not api_key:
      raise Exception("A key should be provided to invoke the endpoint")

    client = ChatCompletionsClient(
        endpoint='https://Phi-3-5-mini-instruct-bzsru.eastus2.models.ai.azure.com',
        credential=AzureKeyCredential(api_key)
    )
    
    result = client.complete({
        "messages": [
          {
            "role": "user",
            "content": "Return a short comma-separated list (less than 5 items) of a few words describing various fields or topics of the following query:" +
              query + ". Include some words and phrases directly from the query."
          }
        ],
        "temperature": 0.8,
        "top_p": 0.95,
    })
    response = str(result.choices[0].message.content).strip()
    
    # Transform the response into a query by making a list from the result
    query_list = []
    last_comma = 0
    while "," in response[last_comma:]:
        comma_pos = response[last_comma:].find(",") + last_comma
        query_list.append(response[last_comma:comma_pos].strip())
        last_comma = comma_pos + 1
    query_list.append(response[last_comma:].strip().replace(".", ""))
    
    # Create the final response
    response = ""
    for i in range(len(query_list)):
        query = query_list[i]
        query = query.replace(" ", "+")
        response += f"({query})"
        if i != len(query_list) - 1:
            response += "|"
    return response

Here's an example of using this method to search both Semantic Scholar and CORE:

In [9]:
from functions import *
text = prompt_query("career development")
print(text)

paper_df = semantic_scholar_query(text, 10)
paper_df

(career+growth)|(professional+development)|(job+advancement)|(skill+enhancement)|(networking+opportunities)


,title,id,abstract,tldr,field
0,The Role of Mentorship and Coaching in Profess...,822fdc4ffd220330c104cf2da95a166eb144821c,This study explores the pivotal roles of mento...,None,Education
1,Fostering Alumni Involvement and Professional ...,2191cdda78df24a582eb6f7438a71c7a22cbf1b2,This study investigates the National Universit...,The National University's Alumni Portal's user...,"Education, Computer Science"
2,Nurturing Learning Agility: Investigating Self...,584552dfe23fd94f1054e740463a06cf50384aea,"In the modern era, the acquisition of learning...",None,"Education, Psychology"
3,Examining International Graduate Students’ Car...,f3338bd702201ab6de203d9aabf7f403a7d76c96,China has become an increasingly popular desti...,None,"Education, Sociology, Economics"
4,Enhancing Networking skills for Professional s...,438aadf12ac885fc8e906aff7908cdbf1348a83c,This theoretical article aims to explore and d...,None,"Business, Education"
5,Impact of strategic management education on pr...,31c82ffbe63bea9828d9e366a9c9e37cc1deea5b,This study explores the impact of strategic ma...,None,"Education, Business"
6,Linking Succession Planning to Nurses’ Perform...,1c11ec20afc26cc3eaa10ffb37af9d0c12fa23be,Background: Succession planning is essential t...,Healthcare organizations may prepare high-pote...,"Business, Medicine"
7,CAREER GROWTH AND DEVELOPMENT: ROLE OF CONTINU...,41c7447b205577df733a2c79515d08738caa58cc,This paper aims to highlight the role of conti...,None,"Education, Economics"
8,English Language Proficiency and Career Opport...,4bb96bb7edb9a266bbc6bc316a4d2fe42cd008cf,This research investigates the crucial role of...,None,"Education, Linguistics"
9,Professional Networking at Gastroenterology Sc...,a6110a1ad449077fd584f96c77783b53f1a9cfaa,None,The authors participated in a live discussion ...,"Medicine, Medicine, Education"


In [8]:
core_query(text.replace("|", "OR"))

,title,abstract,link,text,citation
0,Artificial Intelligence & Machine Learning in ...,"In the 2020s, Artificial Intelligence (AI) has...",https://core.ac.uk/download/551578520.pdf,ISSN: 2658-8455 ...,"EL KABBOURI, Mounime, et al. ""Artificial Intel..."
1,Predicting risk/reward ratio in financial mark...,Financial market forecasting remains a formida...,http://arxiv.org/pdf/2311.09148,Copyright InformationThis work is licensed und...,"Baghshah, Mahdieh Soleymani, et al. ""Predictin..."
2,Machine Learning and its Uses in Business,Machine learning (ML) has emerged as a powerfu...,https://core.ac.uk/download/613698544.pdf,International Journal on Recent and Innovation...,"Jeetesh Kumar Dhruw. ""Machine Learning and its..."
3,Ethical Implications of Predictive Risk Intell...,open access articleThis paper presents a case ...,https://core.ac.uk/download/228181605.pdf,Ethical Implications of Predictive Risk Intel...,"Jiya, Tilimbe. ""Ethical Implications of Predic..."
4,Toward Business Integrity Modeling and Analysi...,Financialization has contributed to economic g...,https://core.ac.uk/download/323053144.pdf,"Citation:Chang, V and Valverde, R and Ramachan...","Chang, V, et al. ""Toward Business Integrity Mo..."
5,Algorithmic trading with cryptocurrencies,"Since its inception in 2009, Bitcoin has gaine...",https://core.ac.uk/download/570972899.pdf,"A Work Project, presented as part of th...","Röhnelt, Tobias. ""Algorithmic trading with cry..."
6,Algorithmic trading with cryptocurrencies - do...,"Since its inception in 2009, Bitcoin has gaine...",https://core.ac.uk/download/541165383.pdf,"A Work Project, presented as part of th...","Schnülle, Tim. ""Algorithmic trading with crypt..."
7,Big Data analysis and Finance: a literature re...,"In this paper, we discuss the exploitation of ...",https://core.ac.uk/download/492988329.pdf,EHEI Journal of Science and Technology Vol....,"Moussi, A.. ""Big Data analysis and Finance: a ..."
8,Regulating by Robot: Administrative Decision M...,Machine-learning algorithms are transforming l...,https://core.ac.uk/download/151696128.pdf,University of Pennsylvania Carey Law School Pe...,"Coglianese, Cary, and David Lehr. ""Regulating ..."
